In [54]:
import numpy as np


In [55]:
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature 
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)
print(reweight_distribution)

<function reweight_distribution at 0x13b474a60>


In [56]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 75004 files belonging to 1 classes.


In [57]:
from tensorflow.keras.layers import TextVectorization

In [58]:
sequence_length = 100
vocab_size = 15000
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

In [59]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1] 
    y = vectorized_sequences[:, 1:] 
    return x, y

In [60]:
lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

In [61]:
from tensorflow.keras import layers 
embed_dim = 256
latent_dim = 2048
num_heads = 2

In [62]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1) 
        embedded_tokens = self.token_embeddings(inputs) 
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    def compute_mask(self, inputs, mask=None): 
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super().get_config() 
        config.update({
            "output_dim": self.output_dim, "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [63]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
    def get_config(self):
        config = super().get_config() 
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [64]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs) 
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

In [65]:
import numpy as np
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

In [66]:
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64") 
    predictions = np.log(predictions) / temperature 
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1) 
    return np.argmax(probas)

In [67]:
class TextGenerator(keras.callbacks.Callback): 
    def __init__(self,prompt, generate_length, model_input_length, temperatures=(1.,), print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq
    def on_epoch_end(self, epoch, logs=None): 
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature) 
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, i, :])
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token 
            print(sentence)


In [68]:
prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

In [69]:
model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Epoch 1/200


ValueError: in user code:

    File "/Users/janswidzinski/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/janswidzinski/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/janswidzinski/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/janswidzinski/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 997, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/Users/janswidzinski/miniforge3/lib/python3.10/site-packages/keras/engine/base_layer.py", line 2221, in trainable_variables
        return self.trainable_weights
    File "/Users/janswidzinski/miniforge3/lib/python3.10/site

In [72]:
bf_train = model.get_weights()
bf_train

ValueError: Weights for model sequential_3 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.

In [71]:
print (model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_3 (Positi  (None, None, 256)   3865600     ['input_4[0][0]']                
 onalEmbedding)                                                                                   
                                                                                                  


ValueError: Weights for model sequential_3 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.